# Chapter 4 - Interest rates

## Imports

In [62]:
import math

## Problems

### 4.1

In [63]:
quarterly_rate = 0.07
quarterly_compounded = math.pow(1 + quarterly_rate / 4, 4)

#### Continuous compounding

In [64]:
continuous_rate = math.log(quarterly_compounded)
print("continuous rate: ", continuous_rate)

continuous rate:  0.06939455333845228


#### Annually compounded

In [65]:
annual_rate = quarterly_compounded - 1
print("annual rate: ", annual_rate)

annual rate:  0.07185903128906279


### 4.2

In [66]:
semi_annual_rate = 0.05
continuous_rate = math.log((1 + semi_annual_rate / 2) ** 2)
print("continuous zero rate: ", continuous_rate)

continuous zero rate:  0.049385225180742925


#### Bond price

In [67]:
bond_yield = 0.052
half_year_yield = math.exp(-bond_yield / 2)
bond_price = 2 * half_year_yield * (1 + half_year_yield) + 102 * (half_year_yield ** 3)
print("bond price: ", bond_price)

bond price:  98.1936994203443


#### 18 month (continuous) zero rate

In [68]:
half_year_cost = math.exp(continuous_rate / 2)
half_year = 2 * half_year_cost
full_year = 2 * half_year_cost ** 2
extended_zero_rate = -math.log((bond_price - half_year - full_year) / 102) / 1.5
print("18 month zero rate: ", extended_zero_rate)

18 month zero rate:  0.0541510289146814


### 4.3

In [69]:
yearly_rate = 0.1
print("yearly rate: ", yearly_rate)
semi_annual_rate = (math.sqrt(1 + yearly_rate) - 1) * 2
print("semi-annual rate: ", semi_annual_rate)
monthly_rate = (math.pow(1 + yearly_rate, 1 / 12) - 1) * 12
print("monthly rate: ", monthly_rate)
continuous_rate = math.log(1 + yearly_rate)
print("continuous rate: ", continuous_rate)

yearly rate:  0.1
semi-annual rate:  0.09761769634030326
monthly rate:  0.09568968514684517
continuous rate:  0.09531017980432493


### 4.4

#### Risk-free rates

In [70]:
three_month_rate = 0.03
print("three month rate: ", three_month_rate)
six_month_rate = 0.032
print("six month rate: ", six_month_rate)
nine_month_rate = 0.034
print("nine month rate: ", nine_month_rate)
twelve_month_rate = 0.035
print("twelve month rate: ", twelve_month_rate)
fifteen_month_rate = 0.036
print("fifteen month rate: ", fifteen_month_rate)
eighteen_month_rate = 0.037
print("eighteen month rate: ", eighteen_month_rate)

three month rate:  0.03
six month rate:  0.032
nine month rate:  0.034
twelve month rate:  0.035
fifteen month rate:  0.036
eighteen month rate:  0.037


#### Forward rates

##### The forward rate calculation

In [71]:
def forward_rate(previous_rate: float, previous_duration: float, next_rate: float, duration_difference: float) -> float:
    '''
    All durations are in years.  duration_difference is the next duration minus the previous duration.  E.g., 
    for the forward rates between quarters, this would be 3 months or 1/4 of a year.
    '''
    return next_rate + (previous_duration / duration_difference) * (next_rate - previous_rate)

print("The results of Table 4.5 are replicated below using forward_rate.")
print("year 1 -> year 2 forward rate: ", forward_rate(0.03, 1, 0.04, 1))
print("year 2 -> year 3 forward rate: ", forward_rate(0.04, 2, 0.046, 1))
print("year 3 -> year 4 forward rate: ", forward_rate(0.046, 3, 0.05, 1))
print("year 4 -> year 5 forward rate: ", forward_rate(0.05, 4, 0.053, 1))  

The results of Table 4.5 are replicated below using forward_rate.
year 1 -> year 2 forward rate:  0.05
year 2 -> year 3 forward rate:  0.057999999999999996
year 3 -> year 4 forward rate:  0.06200000000000001
year 4 -> year 5 forward rate:  0.06499999999999997


##### The quarterly forward rates

In [72]:
quarter_duration = 1 / 4
print("quarter conversion to years: ", quarter_duration)
print("====================================")

print("quarter 2 forward rate: ", forward_rate(three_month_rate, quarter_duration, six_month_rate, quarter_duration))
print("quarter 3 forward rate: ", forward_rate(six_month_rate, 2 * quarter_duration, nine_month_rate, quarter_duration))
print("quarter 4 forward rate: ", forward_rate(nine_month_rate, 3 * quarter_duration, twelve_month_rate, quarter_duration))
print("quarter 5 forward rate: ", forward_rate(twelve_month_rate, 4 * quarter_duration, fifteen_month_rate, quarter_duration))
print("quarter 6 forward rate: ", forward_rate(fifteen_month_rate, 5 * quarter_duration, eighteen_month_rate, quarter_duration))

quarter conversion to years:  0.25
quarter 2 forward rate:  0.034
quarter 3 forward rate:  0.038000000000000006
quarter 4 forward rate:  0.038000000000000006
quarter 5 forward rate:  0.03999999999999997
quarter 6 forward rate:  0.042


### 4.5

#### Convert rates to continuous

In [73]:
continuous_receive_rate = math.log(pow(1 + 0.045/4, 4))
print("4.5% compounded quarterly: ", continuous_receive_rate)

4.5% compounded quarterly:  0.04474875756225748


In [74]:
principal = 1000000
# From previous question, we want the quarter 5 forward rate
agreed_forward_rate = forward_rate(twelve_month_rate, 4 * quarter_duration, fifteen_month_rate, quarter_duration)
print("forward rate between the 12th and 15th months: ", agreed_forward_rate)
FRA_value = principal * (1 / 4) * (continuous_receive_rate - agreed_forward_rate)
print("FRA value to pay 4.5% and receive SOFR on $1,000,000: ", FRA_value)

forward rate between the 12th and 15th months:  0.03999999999999997
FRA value to pay 4.5% and receive SOFR on $1,000,000:  1187.189390564376


### 4.6

If the term structure of interest rates is upward sloping, then the order of magnitudes is as follows from smallest to greatest  
- bond yield on 5-year coupon-bearing bond
- the 5 year zero rate
- the forward rate between 4.75 and 5 years

It's assumed that the upward-sloping nature is not pathologically small or localized at the 5 year mark. The price of the bond will be weighted by the coupon payouts in years 0-4, which are (by definition of upward sloping) lower interest than the 5 year mark when the principal is due.  As a result, the bond yield--calculated with a flat rate--will be lower than the 5 year zero rate.  The forward rate will be increased as it is necessarily larger than two rates involved, given that the interest curve is increasing between the two.  In this case, the two rates are at 4.75 and 5 years in the future, so the forward rate for this period is necessarily larger than the zero rate for 5 years.

#### Flipped curve [Todo]

### 4.7

The duration relates the sensitivity of the value of a bond-like investment to changes in the yield.  Because the yield likely shifts opposite the zero rates over the bond terms, the duration is a measure of how sensitive a bond's value is to change in the zero rates.  

This analysis is based on small variations due to small shifts in the yield (from which the interest rate is inferred).  For large shifts in any of these quantities, their linear relationship is no longer guaranteed; the trend may even be wrong in sign.

### 4.8

In [75]:
continuous_rate = math.log(math.pow((1 + 0.08 / 12), 12))
print("continuous compounding equivalent to 8% per annum compounded monthly: ", continuous_rate)

continuous compounding equivalent to 8% per annum compounded monthly:  0.07973451262402206


### 4.9

In [76]:
rate = 0.04
principal = 10000
quarter_compounded = math.exp(rate / 4)
first_quarter = principal * (quarter_compounded - 1)
print("first quarter payout: ", first_quarter)
second_quarter = (principal + first_quarter) * (quarter_compounded - 1)
print("second quarter payout: ", second_quarter)
third_quarter = (principal + first_quarter + second_quarter) * (quarter_compounded - 1)
print("third quarter payout: ", third_quarter)
fourth_quarter = (principal + first_quarter + second_quarter + third_quarter) * (quarter_compounded - 1)
print("fourth quarter payout: ", fourth_quarter)

first quarter payout:  100.5016708416795
second quarter payout:  101.51172942587641
third quarter payout:  102.53193926760932
fourth quarter payout:  103.56240238871256


### 4.10

In [77]:

principal = 100
coupon = 4
six_month = coupon * math.exp(-0.04 / 2)
twelve_month = coupon * math.exp(-0.042)
eighteen_month = coupon * math.exp(-0.044 * 3 / 2)
twentyfour_month = coupon * math.exp(-0.046 * 2)
thirty_month = (principal + coupon) * math.exp(-0.048 * 5 / 2)
price = six_month + twelve_month + eighteen_month + twentyfour_month + thirty_month
print("bond price estimate: ", price)

bond price estimate:  107.38894328944818


### 4.11

#### Solved using Wolframalpha
8 (exp[-y / 2] + exp[-y] + exp[-3 y / 2] + exp[-2 y] + exp[- 5 y / 2]) + 108 exp[-3 y ] == 104

#### Approximate yield
13.8%

### 4.12


In [78]:
six_month = math.exp(-0.05 / 2)
twelve_month = math.exp(-0.06)
eighteen_month = math.exp(-0.065 * 3 / 2)
twentyfour_month = math.exp(-0.07 * 2)
par_yield = 2 * 100 * (1 - twentyfour_month) / (six_month + twelve_month + eighteen_month + twentyfour_month)
print("par yield (in percent) for 2 year paid semi-annually: ", par_yield)

par yield (in percent) for 2 year paid semi-annually:  7.074077478783004


### 4.13

In [79]:
one_year = 0.02
two_year = 0.03
three_year = 0.037
four_year = 0.042
five_year = 0.045

print("forward rate between 1st and 2nd year: ", forward_rate(one_year, 1, two_year, 1))
print("forward rate between 2nd and 3rd year: ", forward_rate(two_year, 1, three_year, 1))
print("forward rate between 3rd and 4th year: ", forward_rate(three_year, 1, four_year, 1))
print("forward rate between 4th and 5th year: ", forward_rate(four_year, 1, five_year, 1))


forward rate between 1st and 2nd year:  0.039999999999999994
forward rate between 2nd and 3rd year:  0.044
forward rate between 3rd and 4th year:  0.04700000000000001
forward rate between 4th and 5th year:  0.047999999999999994


### 4.14

In [80]:
base = 100
years = 10
coupon_8_price = 90
coupon_4_price = 80
ten_year_rate = -math.log((2 * coupon_4_price - coupon_8_price) / base) / 10
print("ten year zero rate: ", ten_year_rate)

ten year zero rate:  0.03566749439387325


### 4.15

The liquidity preference theory of interest rates assumes that different rates for different loan durations are driven by opposing time preferences between lenders and borrowers.  All things being equal, borrowers are expected to prefer longer terms to lower their refinancing risk if rates go up and lenders are expected to prefer shorter terms to lower their opportunity cost if rates go up.  This results in an inbalanced demand for longer term loans from borrowers and an inbalanced supply for shorter term loans from lenders.  One expects, then, that lenders incentivize shorter-term loans with lower interest rates and lenders can charge higher interest rates for longer-term loans due to the increased demand.

### 4.16

An upward-sloping zero rate curve means that longer maturities pay higher interest rates.  The par yield is defined as the (semi-annual) coupon that compensates for the decreased present-value of the bond's principal at maturity.  These coupons are paid out at earlier times, where the present value is determined by the smaller, nearer-term zero rates between now and maturity.  As a result, the bond yield is partially weighted by rates lower than the one associated with the bond maturity.  

Conversely, a download-sloping zero rate curve means that the bond yield is weighted by higher interest rates for nearer terms than the bond maturity.  As a result, the bond yield would exceed the zero rate at maturity in the case of a downward sloping zero rate.